In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [2]:
df = pd.read_excel('Input.xlsx')

In [3]:
df2 = df.copy()
# df2 = df2.iloc[:5, :2]

In [66]:
df2

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [67]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive"
}


In [68]:
for i in range(0, df2.shape[0]):
    link = df.iloc[i, 1]
    webpage = requests.get(link,headers=headers).text
    soup = BeautifulSoup(webpage, 'lxml')
    try:
        heading = soup.find('h1', class_='entry-title').text
        content = soup.find('div', class_='td-post-content tagdiv-type').text
    except Exception as e:
        print(e)
    with open(f'{df2.iloc[i, 0]}.txt', "w") as f:
        f.writelines(heading)
        f.writelines(content)
        

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'


In [74]:
df2

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [70]:
df2.iloc[0, 0]

'blackassign0001'

In [4]:
positive_score = []
negative_score = []
for i in range(0, df2.shape[0]):
    
    with open(f'{df2.iloc[i, 0]}.txt') as f:
        lines = f.readlines()
    lines = [j.lower() for j in lines]
    df1 = pd.DataFrame(lines)
    df1.rename(columns={0 : 'lines'}, inplace=True)
    df1['lines'] = df1['lines'].str.strip()
    empty_lines = df1[df1['lines']=='']
    df1 = df1.drop(empty_lines.index).reset_index().drop(columns='index')
    def remove_abb(data):
        data = re.sub(r'\b\d{4}\b', '', data)
        data = re.sub(r',', '', data)
        data = re.sub(r'[:\-–\.]', '', data)
        
        return data
    df1['lines'] = df1['lines'].apply(remove_abb)
    def remove_punctuation(text):
        for i in string.punctuation:
            if i in text:
                text.replace(i, '')
        return text

    df1['lines'] = df1['lines'].apply(remove_punctuation)

    def tokenize(data):
        data = word_tokenize(data)
        return data
    df1['lines'] = df1['lines'].apply(tokenize)

    def remove_stopwords(text):
        L = []
        for word in text:
            if word not in stopwords.words('english'):
                L.append(word)
        return L

    df1['lines'] = df1['lines'].apply(remove_stopwords)


    with open('negative-words.txt', 'r', encoding='utf-8', errors='ignore') as f:
        l = f.readlines()
        l = [k.strip() for k in l]
        neg = np.array(l)
    with open('positive-words.txt', encoding='utf-8', errors='ignore') as f:
        l = f.readlines()
        l = [z.strip() for z in l]
        pos = np.array(l)

    def check(word_list):
        pos_count = 0
        neg_count = 0
        for word in word_list:
            if word in pos:
                pos_count += 1
            elif word in neg:
                neg_count += 1
        return neg_count, pos_count

    s = df1['lines'].apply(check).reset_index()
    s['first'] = s['lines'].apply(lambda x: x[0])
    s['second'] = s['lines'].apply(lambda x: x[1])
    positive_score.append(s['first'].sum())
    negative_score.append(s['second'].sum())
# positive_score


In [5]:
df2['Positive_Score'] = positive_score
df2['Negative_Score'] = negative_score


In [76]:
df2

,URL_ID,URL,Positive_Score,Negative_Score
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,6,44
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,31,64
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,24,40
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,75,40
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,8,23
...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,57,30
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,35,32
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,3,7
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,3,7


In [9]:
df2['Polarity_Score'] = (df2['Positive_Score'] - df2['Negative_Score'])/ ((df2['Positive_Score'] + df2['Negative_Score']) + 0.000001)


In [10]:
df2

,URL_ID,URL,Positive_Score,Negative_Score,Polarity_Score
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,6,44,-0.760000
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,31,64,-0.347368
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,24,40,-0.250000
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,75,40,0.304348
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,8,23,-0.483871
...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,57,30,0.310345
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,35,32,0.044776
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,3,7,-0.400000
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,3,7,-0.400000
